In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
"""Provides information about the commitments made by one state to a specific
dyadic partner in a given year. When alliance obligations are asymmetric, the dyad-year and
directed dyad-year data differ"""
atop_directed_year = pd.read_csv("./ATOP_data/atop5_1ddyr.csv")

In [3]:
""" Extra columns

ddyad: two cow codes of dyad combined together
atopally: 1 if they share an alliance (1 for all columns in this dataset)
shareob: 1 if they have no obligations to each other
transyr: 1 if alliance starts and ends in the given year
biatno: total number of bi-lateral alliances for this directed dyad in the given year
multino: total number of multi-lateral alliances for this directed dyad in the given year
number: total number of alliances for this directed dyad in the given year
stateA: cow codes of the states
stateB: cow codes of the states
"""

' Extra columns\n\nddyad: two cow codes of dyad combined together\natopally: 1 if they share an alliance (1 for all columns in this dataset)\nshareob: 1 if they have no obligations to each other\ntransyr: 1 if alliance starts and ends in the given year\nbiatno: total number of bi-lateral alliances for this directed dyad in the given year\nmultino: total number of multi-lateral alliances for this directed dyad in the given year\nnumber: total number of alliances for this directed dyad in the given year\nstateA: cow codes of the states\nstateB: cow codes of the states\n'

In [4]:
atop_directed_year.info()
atop_directed_year.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273296 entries, 0 to 273295
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ddyad     273296 non-null  int64  
 1   year      273296 non-null  int64  
 2   atopally  273296 non-null  int64  
 3   defense   273296 non-null  int64  
 4   offense   273296 non-null  int64  
 5   neutral   273296 non-null  int64  
 6   nonagg    273296 non-null  int64  
 7   consul    273296 non-null  int64  
 8   shareob   258 non-null     float64
 9   transyr   273296 non-null  int64  
 10  bilatno   273296 non-null  int64  
 11  multino   273296 non-null  int64  
 12  number    273296 non-null  int64  
 13  asymm     273296 non-null  int64  
 14  atopid1   273296 non-null  int64  
 15  atopid2   106874 non-null  float64
 16  atopid3   21300 non-null   float64
 17  atopid4   6044 non-null    float64
 18  atopid5   644 non-null     float64
 19  atopid6   252 non-null     float64
 20  atop

,ddyad,year,atopally,defense,offense,neutral,nonagg,consul,shareob,transyr,...,atopid3,atopid4,atopid5,atopid6,atopid7,atopid8,atopid9,stateA,stateB,version
0,2020,1942,1,1,1,0,0,0,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,5.1
1,2020,1943,1,1,1,0,0,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,5.1
2,2020,1944,1,1,1,0,0,0,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,5.1
3,2020,1945,1,1,1,0,0,0,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,5.1
4,2020,1949,1,1,0,0,1,1,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,20,5.1


In [5]:
atop_directed_year.isnull().sum()

ddyad            0
year             0
atopally         0
defense          0
offense          0
neutral          0
nonagg           0
consul           0
shareob     273038
transyr          0
bilatno          0
multino          0
number           0
asymm            0
atopid1          0
atopid2     166422
atopid3     251996
atopid4     267252
atopid5     272652
atopid6     273044
atopid7     273172
atopid8     273246
atopid9     273284
stateA           0
stateB           0
version          0
dtype: int64

In [6]:
"""Number of states"""
print(f"There are {pd.concat([atop_directed_year['stateA'],atop_directed_year['stateB']]).nunique()} states in the dataset")

There are 207 states in the dataset


In [7]:
# Condensing atopid columns 1 to 9 into a single column as a list
atopids = [f'atopid{i}' for i in range(1, 10)]

# Combining into a list
atop_directed_year['atopids'] = atop_directed_year[atopids].apply(lambda x: x.dropna().to_list(), axis=1)

# Dropping the original columns
atop_directed_year.drop(atopids, axis=1, inplace=True)

In [8]:
"""Separating 'ddyad' into 2 columns 'state_nameA' & 'state_nameB' """

# Adding ccodes redundant as we have stateA and stateB
#atop_directed_year['ccode1'] = (atop_directed_year['ddyad'] // 1000)
#atop_directed_year['ccode2'] = (atop_directed_year['ddyad'] % 1000)

# Loading in COW codes
cow_df = pd.read_csv("../COW/COW_data/COW-country-codes.csv")
# CCode to state name series
name_series = pd.Series(cow_df['StateNme'].unique())
name_series.index = cow_df['CCode'].unique()

# Mapping the COW codes to the state names for my own sanity

for i in range(len(atop_directed_year['stateA'])):
    atop_directed_year.at[i, 'state_nameA'] = name_series[atop_directed_year.at[i, 'stateA']]
    atop_directed_year.at[i, 'state_nameB'] = name_series[atop_directed_year.at[i, 'stateB']]

# Reordering
atop_columns = list(atop_directed_year.columns)
atop_columns.remove('state_nameA')
atop_columns.remove('state_nameB')
atop_columns.insert(1, 'state_nameA')
atop_columns.insert(2, 'state_nameB')


atop_directed_year = atop_directed_year[atop_columns]

# Dropping now obsolete ddyad column
atop_directed_year.drop('ddyad', axis=1, inplace=True)
atop_directed_year

,state_nameA,state_nameB,year,atopally,defense,offense,neutral,nonagg,consul,shareob,transyr,bilatno,multino,number,asymm,stateA,stateB,version,atopids
0,United States of America,Canada,1942,1,1,1,0,0,0,NaN,1,0,1,1,0,2,20,5.1,[2550.0]
1,United States of America,Canada,1943,1,1,1,0,0,0,NaN,0,0,1,1,0,2,20,5.1,[2550.0]
2,United States of America,Canada,1944,1,1,1,0,0,0,NaN,0,0,1,1,0,2,20,5.1,[2550.0]
3,United States of America,Canada,1945,1,1,1,0,0,0,NaN,1,0,1,1,0,2,20,5.1,[2550.0]
4,United States of America,Canada,1949,1,1,0,0,1,1,NaN,1,0,1,1,0,2,20,5.1,[3180.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273291,Samoa,China,2014,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273292,Samoa,China,2015,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273293,Samoa,China,2016,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273294,Samoa,China,2017,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]


In [9]:
atop_columns

['ddyad',
 'state_nameA',
 'state_nameB',
 'year',
 'atopally',
 'defense',
 'offense',
 'neutral',
 'nonagg',
 'consul',
 'shareob',
 'transyr',
 'bilatno',
 'multino',
 'number',
 'asymm',
 'stateA',
 'stateB',
 'version',
 'atopids']

In [10]:
atop_directed_year[atop_directed_year['state_nameA'] == 'Djibouti']

,state_nameA,state_nameB,year,atopally,defense,offense,neutral,nonagg,consul,shareob,transyr,bilatno,multino,number,asymm,stateA,stateB,version,atopids
203830,Djibouti,France,1977,1,0,0,0,0,0,NaN,1,1,0,1,1,522,220,5.1,[3790.0]
203831,Djibouti,France,1978,1,0,0,0,0,0,NaN,0,1,0,1,1,522,220,5.1,[3790.0]
203832,Djibouti,France,1979,1,0,0,0,0,0,NaN,0,1,0,1,1,522,220,5.1,[3790.0]
203833,Djibouti,France,1980,1,0,0,0,0,0,NaN,0,1,0,1,1,522,220,5.1,[3790.0]
203834,Djibouti,France,1981,1,0,0,0,0,0,NaN,0,1,0,1,1,522,220,5.1,[3790.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205500,Djibouti,Oman,2014,1,1,0,0,1,1,NaN,0,0,2,2,0,522,698,5.1,"[3015.0, 3205.0]"
205501,Djibouti,Oman,2015,1,1,0,0,1,1,NaN,0,0,2,2,0,522,698,5.1,"[3015.0, 3205.0]"
205502,Djibouti,Oman,2016,1,1,0,0,1,1,NaN,0,0,2,2,0,522,698,5.1,"[3015.0, 3205.0]"
205503,Djibouti,Oman,2017,1,1,0,0,1,1,NaN,0,0,2,2,0,522,698,5.1,"[3015.0, 3205.0]"


In [11]:
""" Finding asymmetrical alliances, really tickled some brain juice out of me """
asymmetrical = atop_directed_year[atop_directed_year['asymm'].astype(bool)]

# Gets all country pair combinations as dataframes
country_pairs = asymmetrical.groupby(['state_nameA', 'state_nameB'])
country_pair_dfs = {name:group for name, group in country_pairs}
# Finding unique pairs

unique_obligations = pd.DataFrame(columns=['states_pair', 'atopids', 'obligation_score'])
# Iterates through each pair of countries dataframe
for name, pair_df in country_pair_dfs.items():
    stateA = name[0]
    stateB = name[1]
    name = f"{stateA} -> {stateB}"
    asym_allies = []
    obligations = []
    # print(pair_df.groupby('atopids').size())
    # Iterates through each year to find a unique combination of atopids
    for i, row in pair_df.iterrows():
        obligation_score = (row['defense'] + row['offense'] + 1)//2 *8 + row['neutral']*4 + row['nonagg']*2 + row['consul']
        alliance_ids = row['atopids']
        #Append if unique atopids combination
        if alliance_ids not in asym_allies:
            asym_allies.append(alliance_ids)
            obligations.append(obligation_score)

    unique_obligations = pd.concat([unique_obligations, pd.DataFrame({'states_pair':name, 'stateA':stateA, 'stateB':stateB, 'atopids':asym_allies, 'obligation_score':obligations})])

uk_case = pd.concat([unique_obligations[unique_obligations['stateA'] == 'United Kingdom'],unique_obligations[unique_obligations['stateB'] == 'United Kingdom']])
uk_case

,states_pair,atopids,obligation_score,stateA,stateB
0,United Kingdom -> Austria-Hungary,[1110.0],8,United Kingdom,Austria-Hungary
0,United Kingdom -> Belgium,[2130.0],8,United Kingdom,Belgium
0,United Kingdom -> China,[1118.0],8,United Kingdom,China
0,United Kingdom -> Cyprus,[3405.0],9,United Kingdom,Cyprus
0,United Kingdom -> Egypt,[2385.0],9,United Kingdom,Egypt
...,...,...,...,...,...
2,Turkey -> United Kingdom,[1330.0],0,Turkey,United Kingdom
3,Turkey -> United Kingdom,"[2420.0, 2490.0]",13,Turkey,United Kingdom
4,Turkey -> United Kingdom,[2490.0],13,Turkey,United Kingdom
5,Turkey -> United Kingdom,"[2490.0, 2550.0]",13,Turkey,United Kingdom


In [12]:
""" Finding significant patron and protege states """
patron_states = {}
protege_states = {}
for i, row in unique_obligations.iterrows():
    # Find the opposite pair
    opposite_pair_name = row['states_pair'].split(' -> ')
    opposite_pair_name = f"{opposite_pair_name[1]} -> {opposite_pair_name[0]}"
    opposite_pair = unique_obligations[unique_obligations['states_pair'] == opposite_pair_name]
    for i, opp_row in opposite_pair.iterrows():
        if opp_row['atopids'] == row['atopids']:
            opposite_pair = opp_row
            break

    # Determine if patron or protege
    stateA = row['states_pair'].split(' -> ')[0]
    stateB = row['states_pair'].split(' -> ')[1]
    if row['obligation_score'] > opposite_pair['obligation_score']:
        patron_states[stateA] = patron_states.get(stateA, 0) + 0.5
        protege_states[stateB] = protege_states.get(stateB, 0) + 0.5
    elif row['obligation_score'] < opposite_pair['obligation_score']:
        patron_states[stateB] = patron_states.get(stateB, 0) + 0.5
        protege_states[stateA] = protege_states.get(stateA, 0) + 0.5

pd.DataFrame(patron_states, index=['count']).T
#pd.DataFrame(protege_states, index=['count']).T

,count
Andorra,2.0
Argentina,1.0
Bavaria,2.0
France,9.0
Hanover,1.0
Hesse Grand Ducal,1.0
Austria-Hungary,2.0
Saxony,1.0
Wuerttemburg,1.0
Canada,1.0


In [13]:
#fav_asym_df = pd.DataFrame(asym_fav,index=['count']).T
#unfav_asym_df = pd.DataFrame(asym_unfav,index=['count']).T
#fav_asym_df
#set1 = set([1,2,3,4,5])
#set2 = set([1,2,3,4,5,6])
#set12 = set([1,2,3,4,5])
#set1 == set12
#max_fav_value = max(asym_fav.values())
#max_fav_key = max(asym_fav, key=asym_fav.get)
#print(f"{max_fav_key} has the most asymmetrical dyads with {max_fav_value}")

In [14]:
#"""Plotting the number of asymmetrical alliances by state"""
## Create dataframe for plot
#asymmetric_df = pd.DataFrame({'state_name':asym_allies.keys(),'alliances':asym_allies.values()})
#asymmetric_df = asymmetric_df.sort_values(by='alliances', ascending=False)
#
## Horizontal bar plot
#plt.figure(figsize=(7, 14))
#plt.barh(asymmetric_df['state_name'], asymmetric_df['alliances'], color='red')
#plt.ylabel('State')
#plt.xlabel('Number of Asymmetrical Alliances')
#plt.title('Number of Asymmetrical Alliances by State')
#plt.tight_layout()
#

In [15]:
#""" Plotting the number of asymmetric alliances by state as a chloropleth map """
#fig = px.choropleth(asymmetric_df, locations='state_name', locationmode='country names', color='alliances', hover_name='state_name',labels={'alliances':'Asymmetrical Alliances'},width=600, height=400)
#fig.show()

In [16]:
""" Exporting data for network """
year = 2018
atop_directed_year[atop_directed_year['year'] == year].to_csv(f'./ATOP_data/atop_directed_{year}.csv', index=False)

In [17]:
""" Exporting data for network modelling """
atop_directed_year.to_csv(f'./ATOP_data/alliance_data.csv', index=False)

In [18]:
atop_directed_year

,state_nameA,state_nameB,year,atopally,defense,offense,neutral,nonagg,consul,shareob,transyr,bilatno,multino,number,asymm,stateA,stateB,version,atopids
0,United States of America,Canada,1942,1,1,1,0,0,0,NaN,1,0,1,1,0,2,20,5.1,[2550.0]
1,United States of America,Canada,1943,1,1,1,0,0,0,NaN,0,0,1,1,0,2,20,5.1,[2550.0]
2,United States of America,Canada,1944,1,1,1,0,0,0,NaN,0,0,1,1,0,2,20,5.1,[2550.0]
3,United States of America,Canada,1945,1,1,1,0,0,0,NaN,1,0,1,1,0,2,20,5.1,[2550.0]
4,United States of America,Canada,1949,1,1,0,0,1,1,NaN,1,0,1,1,0,2,20,5.1,[3180.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273291,Samoa,China,2014,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273292,Samoa,China,2015,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273293,Samoa,China,2016,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
273294,Samoa,China,2017,1,0,0,0,1,0,NaN,0,1,0,1,0,990,710,5.1,[3746.0]
